# 🔢 MNIST Handwritten Digit Recognition

A step-by-step walkthrough of building a **Convolutional Neural Network (CNN)** to classify handwritten digits.

This notebook covers:
1. Understanding the MNIST Dataset
2. Exploring the CNN Architecture
3. Training the Model
4. Making Predictions
5. Visualizing Results

## 📦 Setup & Imports

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Check device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 1️⃣ Understanding the MNIST Dataset

MNIST is a dataset of 70,000 handwritten digit images:
- **60,000** training images
- **10,000** test images
- Each image is **28×28 pixels**, grayscale
- **10 classes** (digits 0-9)

In [ ]:
# Load MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # MNIST mean and std
])

train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, download=True, transform=transform)

print(f"Training samples: {len(train_dataset):,}")
print(f"Test samples: {len(test_dataset):,}")
print(f"Image shape: {train_dataset[0][0].shape}")

### Visualizing Sample Digits

In [ ]:
# Display sample images
fig, axes = plt.subplots(2, 5, figsize=(12, 5))

for i, ax in enumerate(axes.flat):
    image, label = train_dataset[i]
    ax.imshow(image.squeeze(), cmap='gray')
    ax.set_title(f'Label: {label}', fontsize=12)
    ax.axis('off')

plt.suptitle('Sample MNIST Digits', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### Class Distribution

In [ ]:
# Count each digit
labels = [train_dataset[i][1] for i in range(len(train_dataset))]
unique, counts = np.unique(labels, return_counts=True)

plt.figure(figsize=(10, 5))
plt.bar(unique, counts, color='steelblue', edgecolor='black')
plt.xlabel('Digit', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.title('Distribution of Digits in Training Set', fontsize=14)
plt.xticks(range(10))
plt.grid(axis='y', alpha=0.3)
plt.show()

print("Samples per digit:")
for digit, count in zip(unique, counts):
    print(f"  {digit}: {count:,}")

## 2️⃣ CNN Architecture

Our CNN has:
- **3 Convolutional Blocks** (Conv → BatchNorm → ReLU → MaxPool → Dropout)
- **3 Fully Connected Layers** with dropout
- ~300K parameters

In [ ]:
from model import MNISTNet, get_model_summary

# Create model
model = MNISTNet().to(device)

# Print summary
print(get_model_summary(model))

In [ ]:
# Print layer details
print("Model Architecture:")
print("="*50)
for name, layer in model.named_children():
    params = sum(p.numel() for p in layer.parameters())
    print(f"{name:15} | {str(layer.__class__.__name__):20} | {params:,} params")

### Visualizing a Forward Pass

In [ ]:
# Get a sample image
sample_image, sample_label = train_dataset[0]
sample_batch = sample_image.unsqueeze(0).to(device)

print(f"Input shape: {sample_batch.shape}")

# Forward pass
model.eval()
with torch.no_grad():
    output = model(sample_batch)
    probabilities = F.softmax(output, dim=1)
    prediction = output.argmax(dim=1).item()

print(f"Output shape: {output.shape}")
print(f"\nPredicted digit: {prediction}")
print(f"Actual label: {sample_label}")
print(f"\nProbabilities:")
for i, prob in enumerate(probabilities[0]):
    bar = '█' * int(prob * 20)
    print(f"  {i}: {bar} {prob*100:.1f}%")

## 3️⃣ Training the Model

We'll train for a few epochs and visualize the learning process.

In [ ]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print(f"Training batches: {len(train_loader)}")
print(f"Test batches: {len(test_loader)}")

In [ ]:
# Training function
def train_one_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    return total_loss / total, 100. * correct / total

def evaluate(model, test_loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            total_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    return total_loss / total, 100. * correct / total

In [ ]:
# Initialize fresh model
model = MNISTNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 5
history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

print("Starting training...\n")
print(f"{'Epoch':^7} | {'Train Loss':^12} | {'Train Acc':^10} | {'Val Loss':^12} | {'Val Acc':^10}")
print("-" * 60)

for epoch in range(1, epochs + 1):
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = evaluate(model, test_loader, criterion, device)
    
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    print(f"{epoch:^7} | {train_loss:^12.4f} | {train_acc:^10.2f}% | {val_loss:^12.4f} | {val_acc:^10.2f}%")

print("-" * 60)
print(f"\n✅ Final Validation Accuracy: {val_acc:.2f}%")

### Training Curves

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Loss
ax1.plot(history['train_loss'], 'o-', label='Training', color='#2ecc71', linewidth=2)
ax1.plot(history['val_loss'], 's-', label='Validation', color='#e74c3c', linewidth=2)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training & Validation Loss', fontweight='bold')
ax1.legend()
ax1.grid(alpha=0.3)

# Accuracy
ax2.plot(history['train_acc'], 'o-', label='Training', color='#2ecc71', linewidth=2)
ax2.plot(history['val_acc'], 's-', label='Validation', color='#e74c3c', linewidth=2)
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy (%)')
ax2.set_title('Training & Validation Accuracy', fontweight='bold')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 4️⃣ Making Predictions

Let's use our trained model (or load the pre-trained one) to make predictions.

In [ ]:
from predict import MNISTPredictor

# Load the pre-trained model
predictor = MNISTPredictor('./models/mnist_cnn.pth')

In [ ]:
# Get some test samples
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)
images, labels = next(iter(test_loader))

# Make predictions
predictions, confidences, probs = predictor.predict_batch(images)

# Visualize
fig, axes = plt.subplots(4, 4, figsize=(12, 12))

for i, ax in enumerate(axes.flat):
    img = images[i].squeeze().numpy()
    img = img * 0.3081 + 0.1307  # Denormalize
    
    ax.imshow(img, cmap='gray')
    
    correct = predictions[i] == labels[i].item()
    color = '#2ecc71' if correct else '#e74c3c'
    
    ax.set_title(f'Pred: {predictions[i]} | True: {labels[i]}\nConf: {confidences[i]*100:.1f}%',
                 color=color, fontweight='bold')
    ax.axis('off')

plt.suptitle('Model Predictions on Test Set', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

accuracy = sum(predictions == labels.numpy()) / len(labels) * 100
print(f"Batch Accuracy: {accuracy:.1f}%")

## 5️⃣ Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Get predictions on full test set
all_preds = []
all_labels = []

test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

for images, labels in test_loader:
    preds, _, _ = predictor.predict_batch(images)
    all_preds.extend(preds)
    all_labels.extend(labels.numpy())

# Create confusion matrix
cm = confusion_matrix(all_labels, all_preds)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=range(10), yticklabels=range(10))
plt.xlabel('Predicted', fontsize=12)
plt.ylabel('True', fontsize=12)
plt.title('Confusion Matrix', fontsize=14, fontweight='bold')
plt.show()

# Final accuracy
accuracy = sum(np.array(all_preds) == np.array(all_labels)) / len(all_labels) * 100
print(f"\n✅ Test Set Accuracy: {accuracy:.2f}%")

## 6️⃣ Analyzing Misclassifications

In [ ]:
# Find misclassified samples
misclassified_idx = [i for i, (p, l) in enumerate(zip(all_preds, all_labels)) if p != l]

print(f"Total misclassified: {len(misclassified_idx)} out of {len(all_labels)}")

# Show some misclassified examples
fig, axes = plt.subplots(2, 5, figsize=(15, 6))

for i, ax in enumerate(axes.flat):
    if i >= len(misclassified_idx):
        ax.axis('off')
        continue
        
    idx = misclassified_idx[i]
    img = test_dataset[idx][0].squeeze().numpy()
    img = img * 0.3081 + 0.1307
    
    ax.imshow(img, cmap='gray')
    ax.set_title(f'Pred: {all_preds[idx]} | True: {all_labels[idx]}', color='#e74c3c', fontweight='bold')
    ax.axis('off')

plt.suptitle('Misclassified Examples', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 🎉 Summary

In this notebook, we:

1. ✅ **Explored the MNIST dataset** - 70K handwritten digit images
2. ✅ **Built a CNN** - 3 conv layers + 3 FC layers (~300K params)
3. ✅ **Trained the model** - Achieved ~99% accuracy
4. ✅ **Made predictions** - Classified test images
5. ✅ **Analyzed results** - Confusion matrix and misclassifications

### Next Steps
- Try the web interface: `python app.py`
- Batch process images: `python batch_processor.py`
- Experiment with different architectures